# Imports

In [113]:
from rdhyee_utils.bike import Bike, BikeDocument, BikeWindow

from rdhyee_utils.bike.bikeformat import (
    walk_element,
    innerhtml,
    convert_text,
    text_content,
    get_task_list_items,
    cluster_runs,
    keep_clusters,
    namespaces,
    BALLOT_BOX,
    BALLOT_BOX_WITH_X,
    NS,
    wrap_in_list_item,
    rich_text,
    bike_etree_to_panflute,
    generate_id_attribute,
    generate_unique_id_attribute,
    get_bike_doc,
    etree_to_panflute,
    ids
)

from rdhyee_utils.clipboard.macos import GeneralPasteboard, ptypes
from appscript import app, k, its, mactypes

import os
import sys
import random

import sh

from collections import defaultdict
from itertools import chain, islice

from pathlib import Path as P

from io import StringIO

import lxml.etree as ET
from lxml.html import parse, fromstring
from lxml import etree

import json
import panflute as pf
from panflute import (
    Doc, Header, Str, Para, Div, Span, BulletList, ListItem, Plain, Space, OrderedList, HorizontalRule, BlockQuote, Quoted, Note, CodeBlock, Code, Link, Emph, Strikeout, Strong
)

if True:
    # add pandoc to front of path
    os.environ['PATH'] = '/usr/local/pandoc/pandoc-2.19.2/bin:' + os.environ['PATH']
    # pandoc.configure(path='/usr/local/pandoc/pandoc-2.19.2/bin/pandoc')

import pandoc
from pandoc import types as PT

# This might work but I still got the warning message about the too new pandoc version unless I messed with PATH
if False:
    pandoc.configure(path='/usr/local/pandoc/pandoc-2.19.2/bin/pandoc')




# pandoc versions

In [2]:
# pypandoc

if True:
    os.environ['PYPANDOC_PANDOC'] = "/usr/local/pandoc/pandoc-3.0.1/bin/pandoc"

import pypandoc

print(pypandoc.get_pandoc_version())
print(pypandoc.get_pandoc_path())
print(pypandoc.get_pandoc_formats())

3.0.1
/usr/local/pandoc/pandoc-3.0.1/bin/pandoc
(['biblatex', 'bibtex', 'commonmark', 'commonmark_x', 'creole', 'csljson', 'csv', 'docbook', 'docx', 'dokuwiki', 'endnotexml', 'epub', 'fb2', 'gfm', 'haddock', 'html', 'ipynb', 'jats', 'jira', 'json', 'latex', 'man', 'markdown', 'markdown_github', 'markdown_mmd', 'markdown_phpextra', 'markdown_strict', 'mediawiki', 'muse', 'native', 'odt', 'opml', 'org', 'ris', 'rst', 'rtf', 't2t', 'textile', 'tikiwiki', 'tsv', 'twiki', 'vimwiki'], ['asciidoc', 'asciidoctor', 'beamer', 'biblatex', 'bibtex', 'chunkedhtml', 'commonmark', 'commonmark_x', 'context', 'csljson', 'docbook', 'docbook4', 'docbook5', 'docx', 'dokuwiki', 'dzslides', 'epub', 'epub2', 'epub3', 'fb2', 'gfm', 'haddock', 'html', 'html4', 'html5', 'icml', 'ipynb', 'jats', 'jats_archiving', 'jats_articleauthoring', 'jats_publishing', 'jira', 'json', 'latex', 'man', 'markdown', 'markdown_github', 'markdown_mmd', 'markdown_phpextra', 'markdown_strict', 'markua', 'mediawiki', 'ms', 'muse', 'n

In [ ]:
if False:
    pypandoc.download_pandoc()

In [3]:


# panflute
""" 
seems like panflute does a shutil.wich('pandoc') to find pandoc 
that means to change which pandoc is used, you need to change the PATH
"""
if False:
    pf.tools.PANDOC_PATH = "/usr/local/pandoc/pandoc-3.0.1/bin/pandoc"

# print(pf.__version__)
(pf.utils.load_pandoc_version(),
pf.tools.pandoc_version,
pf.tools.PANDOC_PATH
)

(None,
 pandoc 2.19.2
 Compiled with pandoc-types 1.22.2.1, texmath 0.12.5.2, skylighting 0.13,
 citeproc 0.8.0.1, ipynb 0.2, hslua 2.2.1
 Scripting engine: Lua 5.4
 User data directory: /Users/raymondyee/.local/share/pandoc
 Copyright (C) 2006-2022 John MacFarlane. Web:  https://pandoc.org
 This is free software; see the source for copying conditions. There is no
 warranty, not even for merchantability or fitness for a particular purpose.,
 None)

In [4]:
# pandoc library



text = "Hello world!"
doc = pandoc.read(text)
doc

Pandoc(Meta({}), [Para([Str('Hello'), Space(), Str('world!')])])

In [5]:
pandoc._configuration

{'auto': True,
 'path': '/usr/local/pandoc/pandoc-2.19.2/bin/pandoc',
 'version': '2.19.2',
 'pandoc_types_version': '1.22.2.1'}

# pandoc writer

In [ ]:
# make a empty xhtml etree

def ids(etree):
    """
    Return a list of ids of the rows
    """
    return [e.attrib["id"] for e in etree.xpath("//*[@id]")]


etree = ET.Element('html', nsmap=namespaces)

# add a head
head = ET.SubElement(etree, 'head')
meta = ET.SubElement(head, 'meta', attrib={'charset': 'utf-8'})

# add a body
body = ET.SubElement(etree, 'body')

# add a root ul to body
root_ul = ET.SubElement(body, 'ul', attrib={'id': generate_unique_id_attribute(8, ids(etree))})

print(ET.tostring(etree, pretty_print=True, encoding="utf-8", xml_declaration=True).decode('utf-8'))


In [6]:
from pandoc import types as PT
from collections import Counter

# make a list of all entities that would imported if I did from pandoc.types import *

# if isinstance(v, type) and issubclass(v, PT.MetaType)
pandoc_types = [k for k, v in PT.__dict__.items()]


c = Counter([type(v) for k, v in PT.__dict__.items()])
c


Counter({pandoc.types.MetaType: 96,
         type: 9,
         module: 7,
         function: 5,
         str: 4,
         NoneType: 2,
         dict: 2,
         _frozen_importlib_external.SourceFileLoader: 1,
         _frozen_importlib.ModuleSpec: 1})

In [8]:
# PT.__dict__.keys()
meta_types = dict([(k, v) for k, v in PT.__dict__.items() if isinstance(v, PT.MetaType)])
abstract_types = dict([(k, v) for k, v in meta_types.items() if not issubclass(v, PT.Constructor)])
concrete_types = dict([(k, v) for k, v in meta_types.items() if issubclass(v, PT.Constructor)])
len(meta_types), len(abstract_types)

# abstract_types.keys()
concrete_types.keys()

dict_keys(['Constructor', 'Pandoc', 'Meta', 'MetaMap', 'MetaList', 'MetaBool', 'MetaString', 'MetaInlines', 'MetaBlocks', 'DefaultStyle', 'Example', 'Decimal', 'LowerRoman', 'UpperRoman', 'LowerAlpha', 'UpperAlpha', 'DefaultDelim', 'Period', 'OneParen', 'TwoParens', 'Format', 'RowHeadColumns', 'AlignLeft', 'AlignRight', 'AlignCenter', 'AlignDefault', 'ColWidth_', 'ColWidthDefault', 'Row', 'TableHead', 'TableBody', 'TableFoot', 'Caption', 'Cell', 'RowSpan', 'ColSpan', 'Plain', 'Para', 'LineBlock', 'CodeBlock', 'RawBlock', 'BlockQuote', 'OrderedList', 'BulletList', 'DefinitionList', 'Header', 'HorizontalRule', 'Table', 'Div', 'Null', 'SingleQuote', 'DoubleQuote', 'DisplayMath', 'InlineMath', 'Str', 'Emph', 'Underline', 'Strong', 'Strikeout', 'Superscript', 'Subscript', 'SmallCaps', 'Quoted', 'Cite', 'Code', 'Space', 'SoftBreak', 'LineBreak', 'Math', 'RawInline', 'Link', 'Image', 'Note', 'Span', 'Citation', 'AuthorInText', 'SuppressAuthor', 'NormalCitation'])

In [9]:
set(abstract_types.keys())

{'Alignment',
 'Attr',
 'Block',
 'CitationMode',
 'ColSpec',
 'ColWidth',
 'Data',
 'Inline',
 'ListAttributes',
 'ListNumberDelim',
 'ListNumberStyle',
 'MathType',
 'MetaValue',
 'QuoteType',
 'ShortCaption',
 'Target',
 'Type',
 'TypeDef'}

In [10]:
import pandas as pd
from pandas import Series, DataFrame

def mro_names(v):
    # return only ones that intersect with the abstract types
    return set([c.__name__ for c in v.mro()]) & set(abstract_types.keys())

concrete_mros = dict([(k, mro_names(v)) for k, v in concrete_types.items()])
# create a DataFrame indexed on the concrete types with columns of all the possible mro names

all_mros = set()
for mros in concrete_mros.values():
    all_mros.update(mros)

# create a dictionary of dictionaries with True or False values
data = {k: {mro: mro in v for mro in all_mros} for k, v in concrete_mros.items()}

# create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(data, orient='index')

# print the DataFrame
print(df)

                Data  ListNumberDelim  MetaValue  QuoteType  Inline  \
Constructor     True            False      False      False   False   
Pandoc          True            False      False      False   False   
Meta            True            False      False      False   False   
MetaMap         True            False       True      False   False   
MetaList        True            False       True      False   False   
...              ...              ...        ...        ...     ...   
Span            True            False      False      False    True   
Citation        True            False      False      False   False   
AuthorInText    True            False      False      False   False   
SuppressAuthor  True            False      False      False   False   
NormalCitation  True            False      False      False   False   

                ListNumberStyle  Alignment  MathType  ColWidth  Block  Type  \
Constructor               False      False     False     False  Fals

In [11]:
df.sum()

Data               78
ListNumberDelim     4
MetaValue           6
QuoteType           2
Inline             20
ListNumberStyle     7
Alignment           4
MathType            2
ColWidth            2
Block              14
Type               78
CitationMode        3
dtype: int64

BulletList([[Block]])

In [39]:
df[df['Block']]

,Data,ListNumberDelim,MetaValue,QuoteType,Inline,ListNumberStyle,Alignment,MathType,ColWidth,Block,Type,CitationMode
Plain,True,False,False,False,False,False,False,False,False,True,True,False
Para,True,False,False,False,False,False,False,False,False,True,True,False
LineBlock,True,False,False,False,False,False,False,False,False,True,True,False
CodeBlock,True,False,False,False,False,False,False,False,False,True,True,False
RawBlock,True,False,False,False,False,False,False,False,False,True,True,False
BlockQuote,True,False,False,False,False,False,False,False,False,True,True,False
OrderedList,True,False,False,False,False,False,False,False,False,True,True,False
BulletList,True,False,False,False,False,False,False,False,False,True,True,False
DefinitionList,True,False,False,False,False,False,False,False,False,True,True,False
Header,True,False,False,False,False,False,False,False,False,True,True,False


In [13]:
# let's try to understand how panflute and pandoc-python enforce the type system for pandoc

text = "Hello world!"

# panflute
# 
pfd = pf.convert_text(text, input_format='markdown', output_format='panflute')
pfd

[Para(Str(Hello) Space Str(world!))]

In [14]:
# https://github.com/boisgera/pandoc/blob/master/src/pandoc/__init__.py#L151
pd = pandoc.read(text)
pandoc.write(doc, format="json")


'{"pandoc-api-version": [1, 22, 2, 1], "meta": {}, "blocks": [{"t": "Para", "c": [{"t": "Str", "c": "Hello"}, {"t": "Space"}, {"t": "Str", "c": "world!"}]}]}'

In [15]:
pd = PT.Pandoc(PT.Meta({}), [PT.Para([PT.Str("Hello world!")])])
pandoc.write(pd, format="json")

'{"pandoc-api-version": [1, 22, 2, 1], "meta": {}, "blocks": [{"t": "Para", "c": [{"t": "Str", "c": "Hello world!"}]}]}'

In [19]:
# let me try to create a Pandoc object with a list of blocks that are not valid
# I don't think pandoc-python enforces the type system
pd = PT.Pandoc(PT.Meta({}), [PT.Str("Hello world!")])
pjson = pandoc.write(pd, format="json")
pjson

# read pjson in with panflute
try:
    pfd = pf.convert_text(pjson, input_format='json', output_format='panflute')
except Exception as e:
    print(e)


JSON parse error: Error in $.blocks[0]: When parsing Text.Pandoc.Definition.Block expected an Object with a tag field where the value is one of [Plain, Para, LineBlock, CodeBlock, RawBlock, BlockQuote, OrderedList, BulletList, DefinitionList, Header, HorizontalRule, Table, Div, Null], but got Str.



In [29]:
# grab selected Bike etree to convert to panflute

def bike_selected_to_pfd_pd(heading_level):
    bike = Bike()

    d = bike.windows[0].document
    etree = d.lxml_etree(from_=d.selection_rows)

    pfd = etree_to_panflute(etree, only_doc_children=True)
    pd = pandoc.read(json.dumps(pfd.to_json()), format="json")

    return (pfd, pd)



In [84]:
(pfd, pd) = bike_selected_to_pfd_pd(heading_level=1)



In [44]:
issubclass(pf.Para, pf.Inline)

False

In [62]:
pfd.content[0].tag

'Header'

In [76]:
for e in pfd.content[1].content:
    print(e.tag)

content_type(pfd.content[1])

ListItem
ListItem
ListItem
ListItem
ListItem


('mixed', set())

In [103]:
# etree = ET.fromstring("<p/>")
etree = ET.Element('p')
# etree.text = "cookies & cream"
etree.append( ET.fromstring("<em>hi</em>") )
etree.append( ET.fromstring("<strong>world</strong>") )

# set the context of p to <em>hi</em> with etree
ET.tostring(etree, pretty_print=True, encoding="utf-8", xml_declaration=True).decode('utf-8')   

"<?xml version='1.0' encoding='utf-8'?>\n<p>\n  <em>hi</em>\n  <strong>world</strong>\n</p>\n"

In [ ]:
# test whether the content elements of e are Block elements or Inline elements or MetaValue

def no_content(e):
    return not hasattr(e, 'content')

def is_block(e):
    if hasattr(e, 'content'):
        return all([issubclass(type(c), pf.Block) for c in e.content])
    else:
        return False

def is_inline(e):
    if hasattr(e, 'content'):
        return all([issubclass(type(c), pf.Inline) for c in e.content])
    else:
        return False
    
def is_inline_or_contentless(e):
    if hasattr(e, 'content'):
        return all([issubclass(type(c), pf.Inline) for c in e.content])
    else:
        return True
    
def is_metavalue(e):
    if hasattr(e, 'content'):
        return all([issubclass(type(c), pf.MetaValue) for c in e.content])
    else:
        return False

def one_and_only_one_true(bool_list):
    true_count = sum(bool_list)
    return true_count == 1

# returns one of "none" , "block", "inline", "metavalue"
def content_type(e):
    type_ = set()
    if no_content(e):
        return "none"
    if is_block(e):
        type_.add("block")
    if is_inline(e):
        type_.add("inline")
    if is_metavalue(e):
        type_.add("metavalue")
    if len(type_) == 1:
        return type_.pop()
    else:
        return ("mixed", type_)
    

def other_h(e):
    return ("other_h", e.tag, is_inline_or_contentless(e))

def Doc_h(e):
    return ("Doc", len(e.content))

def Header_h(e):
    return ("Header", e.level, e.identifier, e.classes, e.attributes, pf.stringify(e), is_inline_or_contentless(e))

def Para_h(e):
    return ("Para", pf.stringify(e), is_inline_or_contentless(e))

def inline_h(e):
    """TO DO: handle rich text later"""
    return (e.tag, pf.stringify(e))

e2f = {
    'Doc': Doc_h,
    'Header': Header_h,
    "Para": Para_h
}

def empty_bike_etree() -> ET.Element:
    etree = ET.Element("html", nsmap=namespaces)

    # add a head
    head = ET.SubElement(etree, "head")
    meta = ET.SubElement(head, "meta", attrib={"charset": "utf-8"})

    # add a body
    body = ET.SubElement(etree, "body")

    # add a root ul to body
    root_ul = ET.SubElement(
        body, "ul", attrib={"id": generate_unique_id_attribute(8, ids(etree))}
    )

    return etree

def add_ids_to_bike_etree(etree):
    """ to the root ul, add ids to all the li elements if ids are not present """ 
    # get root ul of etree
    root_ul = etree.xpath("//ul")[0]
    if root_ul.attrib.get("id") is None:
        root_ul.attrib["id"] = generate_unique_id_attribute(8, ids(etree))

    # get all the li elements
    li_elems = root_ul.xpath(".//li")
    for li_elem in li_elems:
        if li_elem.attrib.get("id") is None:
            li_elem.attrib["id"] = generate_unique_id_attribute(5, ids(etree))

    return etree
    

def panflute_to_bike_etree(pfe) -> ET.Element:
    """
    pfe: panflute element    
    base_header_level: the base header level to use for the document
    TO DO: handle different header levels -- might need to abandon this approach in favor of non-recursive approach with a manual stack
    """

    if is_inline_or_contentless(pfe):
        p_elem = ET.Element("p")
        p_elem.text = pf.stringify(pfe).strip()
        return p_elem
    else:
        if pfe.tag == "Doc":
            etree = empty_bike_etree()
            ul_elem = etree.xpath("//ul")[0]
        else:
            ul_elem = ET.Element("ul")

        try:
            for c in pfe.content:
                e = panflute_to_bike_etree(c)
                li_elem = ET.SubElement(ul_elem, "li")
                li_elem.append(e)
        except AttributeError:
            pass
        else:
            if pfe.tag == "Doc":
                etree = add_ids_to_bike_etree(etree)
                return etree
            else:
                return ul_elem




    # print(ET.tostring(etree, pretty_print=True, encoding="utf-8", xml_declaration=True).decode('utf-8'))



def walk_pfd(e, level=0):
    if is_inline_or_contentless(e):
        print("  " * level, inline_h(e))
    else:
        f = e2f.get(e.tag, other_h)
        print("  " * level, e.tag, f,  f(e))
        try:
            for c in e.content:
                walk_pfd(c, level + 1)
        except AttributeError:
            pass

walk_pfd(pfd)


# print(pf.stringify(pfd))

In [165]:
def panflute_to_bike_etree(pfe) -> ET.Element:
    """
    pfe: panflute element    
    base_header_level: the base header level to use for the document
    TO DO: handle different header levels -- might need to abandon this approach in favor of non-recursive approach with a manual stack
    """

    if is_inline_or_contentless(pfe):
        p_elem = ET.Element("p")
        p_elem.text = pf.stringify(pfe).strip()
        return p_elem
    else:
        if pfe.tag == "Doc":
            etree = empty_bike_etree()
            ul_elem = etree.xpath("//ul")[0]
        else:
            ul_elem = ET.Element("ul")

        try:
            for c in pfe.content:
                e = panflute_to_bike_etree(c)
                li_elem = ET.SubElement(ul_elem, "li")
                li_elem.append(e)
        except AttributeError:
            pass
        else:
            if pfe.tag == "Doc":
                etree = add_ids_to_bike_etree(etree)
                return etree
            else:
                return ul_elem




In [134]:
out_path = P().home() / "obsidian" / "MainRY" / "bike" / "test_20231010.bike"
out_path

PosixPath('/Users/raymondyee/obsidian/MainRY/bike/test_20231010.bike')

In [204]:
walk_element(pfd, level=0)

 Doc
   Header
     Str
   Para
     Str


In [234]:
def walk_element_nr(e, level=0):
    """ non-recursive walk of an element """
    stack = [(e, level)]
    while stack:
        (e, level) = stack.pop()
        print("  " * level, e.tag)
        try:
            for c in reversed(e.content):
                stack.append((c, level + 1))
        except AttributeError:
            pass


def find_ul_ancestor(e, max_level):
    ancestor = e.getparent()
    while ancestor is not None:
        if ancestor.tag == "ul":
            data_level = ancestor.get('data-level', 0)
            if int(data_level) <= max_level:
                return ancestor
        ancestor = ancestor.getparent()
    return None


def panflute_to_bike_etree_nr(e, level=0) -> ET.Element:

    # questions about how header levels are handled as we put them into the etree
    etree = None
    stack = [(e, level)]
    # current ul_elem
    ul_elem = None
    heading_level = 0


    while stack:
        (e, level) = stack.pop()
        print("  " * level, e.tag)

        if is_inline_or_contentless(e):

            li_elem = ET.Element("li")

            # TO DO: handle rich text
            p_elem = ET.SubElement(li_elem, "p")
            p_elem.text = pf.stringify(e).strip()

            # 2 things to figure out: parent_ul (where to attach li_elem) and what the current ul_elem is

            if e.tag == "Header":
                print ("header", e.level, e.identifier, e.classes, e.attributes, pf.stringify(e))
                li_elem.attrib["data-type"] = "heading"
                li_elem.attrib["data-level"] = str(e.level)

                if e.level > heading_level:
                    # child
                    parent_ul = ul_elem
                    print ("e.level > heading_level", ul_elem)
                elif e.level == heading_level:
                    # sibling
                    # parent_ul has to be ul parent of ul_elem                    
                    parent_ul = ul_elem.getparent().getparent()
                else:
                    # e.level < heading_level
                    # uncle or higher
                    parent_ul = find_ul_ancestor(ul_elem, int(e.level)-1)

                heading_level = e.level
                ul_elem = ET.SubElement(li_elem, "ul")
                parent_ul.append(li_elem)
            else:
                ul_elem.append(li_elem)

                
        else:
            if e.tag == "Doc":
                etree = empty_bike_etree()
                ul_elem = etree.xpath("//ul")[0]
            else:
                # BulletList
                # OrderedList
                # ListItem
                print("block", e.tag)

            try:
                for c in reversed(e.content):
                    stack.append((c, level + 1))
            except AttributeError:
                pass
    
    # clean up etree by adding ids
    etree = add_ids_to_bike_etree(etree)
    return etree



In [236]:
(pfd, pd) = bike_selected_to_pfd_pd(heading_level=1)
# etree = panflute_to_bike_etree(pfd)
# etree = panflute_to_bike_etree_non_recursive(pfd)
etree = panflute_to_bike_etree_nr(pfd)


with out_path.open("w") as f:
    f.write(ET.tostring(etree, pretty_print=True, encoding="utf-8", xml_declaration=True).decode('utf-8'))



 Doc
   Header
header 1  [] {} 2023.10.09
e.level > heading_level <Element ul at 0x12da12980>
   Header
header 2  [] {} Daily Flow
e.level > heading_level <Element ul at 0x12da13c80>
   Para
   Para
   Header
header 2  [] {} MyInfoCamp
   Para
   Para
   BulletList
block BulletList
     ListItem
block ListItem
       Plain
     ListItem
block ListItem
       Plain
   Header
header 2  [] {} Serious Adulting
   Header
header 3  [] {} Covered California 2023-2024
e.level > heading_level <Element ul at 0x12d578180>
   BulletList
block BulletList
     ListItem
block ListItem
       Plain
     ListItem
block ListItem
       Plain
     ListItem
block ListItem
       Para
     ListItem
block ListItem
       Para
     ListItem
block ListItem
       Para
     ListItem
block ListItem
       Para
     ListItem
block ListItem
       Para
     ListItem
block ListItem
       Para
     ListItem
block ListItem
       Para
     ListItem
block ListItem
       Para
     ListItem
block ListItem
       Para

In [158]:
# walk_element_nr(pfd)

def prepare(doc):
    pass

def action(elem, doc):
    if isinstance(elem, pf.Element):
        print(elem.tag, elem.index)

# Usage
pf.run_filter(action, prepare=prepare, doc=pfd)


MetaMap None
Str 0
Header 0
Str 0
Para 1
Doc None


Doc

In [162]:
pfd.walk(action)

MetaMap None
Str 0
Header 0
Str 0
Para 1
Doc None


Doc

In [ ]:
issubclass(PT.Para, PT.Block)

In [ ]:
issubclass(PT.Para, PT.Constructor), issubclass(PT.Block, PT.Constructor), type(PT.Block), type(PT.Inline), type(PT.Para)

In [ ]:
p = P.home() / "obsidian" / "MainRY" / "bike" / "test_20230915.bike"


def sort_rows(r):
    sort_order = sorted([(x.id.get(), x.name.get()) for x in r.rows()], key=lambda x: int(x[1]), reverse=False)

    for (id_, name) in sort_order:
        r_from = bike_doc.rawdoc.rows[its.id == id_].get()[0]
        r.move(r_from,to=r.rows.end)


if p.exists():
    # open the file
    bike.open(p)
    bike_doc = bike.document_by_path(p) # type: BikeDocument
else:
    bike_doc = BikeDocument(bike)
    bike_doc.save(p)

# bike_doc.rawdoc.rows.delete()
# see whether I can get a row named "Daily Notes"
# check that there is one and only one row named "Daily Notes"

r = bike_doc.rawdoc.rows[its.name == "Daily Notes"].get()
if len(r) == 0:
    r = bike_doc.rawdoc.make(new=k.row, with_properties={k.name: "Daily Notes"})
elif len(r) > 1:
    raise Exception("more than one row named Daily Notes")
else:
    r = r[0]

r.type.set(k.heading)
r.rows.delete()

# create a list of 15 random numbers between 0 and 100 under the Daily Notes row
for i in range(15):
    r.make(new=k.row, with_properties={k.name: str(random.randint(0,1000))})
    
sort_rows(r)



In [ ]:
r.rows

In [ ]:
# get id, name of children


In [ ]:
# bike_doc.rawdoc.rows[its.name == "Daily Notes"].get()
r_from = bike_doc.rawdoc.rows[its.id == "2bk"].get()[0]
r_from

In [ ]:
# do a manual move of a row
# move the first item to the end
# r.move(r.rows[1], to=r.rows.end)
r.move(r_from,to=r.rows.end)

In [ ]:
r.rows[0].get(), r.rows[1].get(), r.rows[2].get()

In [ ]:
bike_doc.ids

In [ ]:

# adding a row by manipulating the DOM
# create a new li
new_li = etree.Element("li")
# create an id for the li
new_li.attrib["id"] = generate_unique_id_attribute(4, bike_doc.ids)
# no text in the li
# create a new p
new_p = etree.Element("p")
new_p.text = "new p"
# append the p to the li
new_li.append(new_p)

# append to xhtml
# is root_url a reference to the original DOM or value?
root_ul.append(new_li)
# replace the root ul with the modified one in xhtml




In [ ]:
print(ET.tostring(xhtml, pretty_print=True).decode("utf-8"))

In [ ]:
# from copy import deepcopy
# how to copy xhtml etree object
# xhtml_copy = deepcopy(xhtml)

xhtml = bike_doc.lxml_etree()



In [ ]:
def reimport_bike(bike_doc, tranformer=None):
    # source = bike_doc.rawdoc.export(as_=k.bike_format)
    source = bike_doc.lxml_etree()
    # opportunity to transform the source
    if tranformer:
        source = tranformer(source)

    # delete all rows
    bike_doc.rawdoc.rows.delete()
    bike_doc.rawdoc.import_(from_=ET.tostring(source, pretty_print=True).decode("utf-8"),
                       # to=bike_doc.root_row.rawrow.after, 
                       as_=k.bike_format)
    

def add_ryid(xhtml):

    context = ET.iterwalk(xhtml, events=('start',))
    for action, elem in context:
        if elem.tag == f'{NS}li':
            elem.attrib["ryid"] = "" 
    return xhtml
 



In [ ]:
bike_doc.file

In [ ]:
reimport_bike(bike_doc, tranformer=add_ryid)
bike_doc.save()

In [ ]:
import os
from lxml import etree

# Your original XML document
original_xml = '''<root>
    <setting name="CorsTrustedOrigins" serializeAs="Xml">
        <value>
            <ArrayOfString>
                <string>http://example.com</string>
            </ArrayOfString>
        </value>
    </setting>
</root>'''

# The new XML element you want to replace the current xpath'ed element with
new_xml = '''<setting name="CorsTrustedOrigins" serializeAs="Xml">
    <value>
        <ArrayOfString>
            <string>http://maddafakka</string>
        </ArrayOfString>
    </value>
</setting>'''

# Parse the original XML document
tree = etree.fromstring(original_xml)

# Find the element you want to replace
value = tree.find('.//ArrayOfString')

# Parse the new XML element
new_tree = etree.fromstring(new_xml)

# Replace the old element with the new one
value.getparent().replace(value, new_tree.find('.//ArrayOfString'))

# Print the modified XML document
print(etree.tostring(tree, pretty_print=True).decode('utf-8'))

In [ ]:
ET.tostring(value)

In [ ]:
print(ET.tostring(root_ul, pretty_print=True).decode("utf-8"))